In [ ]:
import pandas as pd

#数据加载
dataset=pd.read_csv('./SupplyChain.csv', encoding='unicode_escape') #如果utf-8报错，则加上encoding='unicode_escape'忽略编码问题
dataset.shape
temp=dataset.isnull().sum()
temp[temp>0]

#供应链数据探索
dataset[['Customer Fname', 'Customer Lname']]
#将firstname与lastname进行合并
dataset['Customer Full Name']=dataset['Customer Fname']+dataset['Customer Lname']
dataset[['Customer Full Name', 'Customer Fname', 'Customer Lname']]

dataset['Customer Zipcode'].value_counts()
dataset['Customer Zipcode'].isnull().sum()
#用0进行填充
dataset['Customer Zipcode']=dataset['Customer Zipcode'].fillna(0)
dataset['Customer Zipcode'].isnull().sum()

import matplotlib.pyplot as plt
import seaborn as sns
#特征字段之间相关性，高相关性的特征取一个就好
data=dataset
data.corr() #创建相关矩阵
plt.figure(figsize=(20,10))
sns.heatmap(data.corr(),  annot=True, cmap='collwarm') #annot=True则在图中标记相关性具体数值

#按照不同的Market、Order Region看sales
#基于Market
dataset['Market'].value_counts()
market=data.groupby('Market')
market['Sales per customer'].sum().sort_values(ascending=False).plot.bar(figsize=(12, 6), title='Sales in different markets')
#基于Order Region
region=data.groupby('Order Region')
region['Sales per customer'].sum().sort_values(ascending=False).plot.bar(figsize=(12, 6), title='Sales in different regions')
#按照不同的Category Name
cat=data.groupby('Category Name')
cat['Sales per customer'].sum().sort_values(ascending=False).plot.bar(figsize=(12, 6), title='Salesin different categories')
cat['Sales per customer'].mean().sort_values(ascending=False).plot.bar(figsize=(12, 6), title='Salesin different categories')
#按照不同的时间维度趋势
data['order date (DateOrders)'].describe() #时间是object类型
temp=pd.DatetimeIndex(data['order date (DateOrders)']) #将时间转换为DatetimeIndex对象
#order date (DateOrders)字段中的year, month, weekday, hour, month_year
data['order_year'] = temp.year
data['order_month'] = temp.month
data['order_week_day'] = temp.weekday
data['order_hour'] = temp.hour
data['order_month_year'] = temp.to_period('M') #.to_period('M')表示到month为止，即展示年-月，也可以为'H'、'D'
data[['order_year', 'order_month', 'order_week_day', 'order_hour', 'order_month_year']]
data['order_week_day'].min()
data['order_week_day'].max()
#对销售额进行探索，按照不同时间维度：年、星期、小时、月
plt.figure(figsize=(10, 12))
plt.subplot(4, 2, 1)
df_year=data.groupby('order_year')
df_year['Sales'].mean().plot(figsize=(12, 12), title='Mean Sales in years')
plt.subplot(4, 2, 2)
df_day=data.groupby('order_week_day')
df_day['Sales'].mean().plot(figsize=(12, 12), title='Mean Sales in week days')
plt.subplot(4, 2, 3)
df_hour=data.groupby('order_hour')
df_hour['Sales'].mean().plot(figsize=(12, 12), title='Mean Sales in hours')
plt.subplot(4, 2, 4)
df_month=data.groupby('order_month')
df_month['Sales'].mean().plot(figsize=(12, 12), title='Mean Sales in months')

#Product Price与Sales per customer相关性如何
data.plot(x='Product Price', y='Sales per customer', kind='scatter') #绘制散点图
plt.title('Relationship between Product Price and Sales per customer')
plt.xlabel('Product Price')
plt.ylabel('Sales per customer')

#对用户进行分层运营
#时间类型转换
data['order date (DateOrders)']=pd.to_datetime(data['order date (DateOrders)'])
#统计最后一笔订单时间
data['order date (DateOrders)'].max()

#假设现在是2018-2-1
import datetime
present = datetime.datetime(2018, 2, 1)

#计算每个用户的RFM指标
#按照Order Customer Id进行聚合
customer_seg=data.groupby('Order Customer Id').agg({'order date (DateOrders)': lambda x:(present-x.max()).days, 'Order Id': lambda x: len(x), 'Sales': lambda x: x.sum()})

#将RFM数据划分为4个尺度
quantiles=customer_seg.quantile(q=[0.25, 0.5, 0.75])
quantiles=quantiles.to_dict()

#将字段名称改为R_Value, F_Value, M_Value
customer_seg.rename(columns={'order date (DateOrders)': 'R_Value', 'Order Id': 'F_Value', 'Sales': 'M_Value'}, inplace=True)

#将Value转换为Score
#Recency，越小越好
def R_Score(a, b, c):
    if a <= c[b][0.25]:
        return 4
    if a <= c[b][0.50]:
        return 3
    if a <= c[b][0.75]:
        return 2
    return 1
#Frequency, Money，越大越好
def FM_Score(a, b, c):
    if a <= c[b][0.25]:
        return 1
    if a <= c[b][0.50]:
        return 2
    if a <= c[b][0.75]:
        return 3
    return 4

customer_seg['R_Score']=customer_seg['R_Value'].apply(R_Score, args=('R_Value', quantiles))
customer_seg['F_Score']=customer_seg['F_Value'].apply(R_Score, args=('F_Value', quantiles))
customer_seg['M_Score']=customer_seg['M_Value'].apply(R_Score, args=('M_Value', quantiles))

#计算RFM用户分层
def RFM_User(df):
    if df['M_Score'] > 2 and df['F_Score'] > 2 and df['R_Score'] >2:
        return '重要价值用户'
    if df['M_Score'] > 2 and df['F_Score'] <= 2 and df['R_Score'] >2:
        return '重要发展用户'
    if df['M_Score'] > 2 and df['F_Score'] > 2 and df['R_Score'] <=2:
        return '重要保持用户'
    if df['M_Score'] > 2 and df['F_Score'] <= 2 and df['R_Score'] <=2:
        return '重要挽留用户'
    if df['M_Score'] <= 2 and df['F_Score'] > 2 and df['R_Score'] >2:
        return '一般价值用户'
    if df['M_Score'] <= 2 and df['F_Score'] <= 2 and df['R_Score'] >2:
        return '一般发展用户'
    if df['M_Score'] <= 2 and df['F_Score'] > 2 and df['R_Score'] <=2:
        return '一般保持用户'
    if df['M_Score'] <= 2 and df['F_Score'] <= 2 and df['R_Score'] <=2:
        return '一般挽留用户'

customer_seg['Customer_Segmentation']=customer_seg.apply(RFM_User, axis=1)

#显示不同地区的支付类型情况
data['Type'].value_count()
pay_type1=data[data['Type']=='DEBIT']
pay_type2=data[data['Type']=='TRANSFER']
pay_type3=data[data['Type']=='PAYMENT']
pay_type4=data[data['Type']=='CASH']

#获取4种支付方式中，不同地区的特点
count1=pay_type1['Order Region'].value_counts() #DEBIT
count2=pay_type2['Order Region'].value_counts() #TRANSFER
count3=pay_type3['Order Region'].value_counts() #PAYMENT
count4=pay_type4['Order Region'].value_counts() #CASH

#获取地区region的个数
region_num=len(count1)
fig, ax = plt.subplots(figsize=(20, 8))
index=np.arange(region_num)
#每种类型的显示间隔
bar_width=0.2
type1=plt.bar(index, count1, bar_width, color='b', label='DEBIT')
type2=plt.bar(index+bar_width, count2, bar_width, color='r', label='TRANSFER') #+bar_width能够做出多bar对比展示的效果
type3=plt.bar(index+bar_width*2, count3, bar_width, color='g', label='PAYMENT')
type4=plt.bar(index+bar_width*3, count4, bar_width, color='y', label='CASH')
plt.xlabel('Order Region')
plt.ylabel('Number of payments')
plt.title('Type of payments in all regions')
plt.legend()
#显示刻度
names=data['Order Region'].value_counts().keys()
plt.xticks(index+bar_width, names, rotation='vertical')
plt.show()

'''
分析结论
DEBIT支付是所有地区中使用次数最多的方式
CASH支付是所有地区中使用次数最少的方式
'''

#对于欺诈订单和迟交货订单进行预测
#对负收益产品进行探索
loss=data[data['Benefit per order'] < 0]
#显示top10的负收益产品并用bar可视化
loss['Category Name'].value_counts().nlargest(10).plot.bar(figsize=(20, 8), title='Products with most loss')

#显示Top10的负收益地区
loss['Order Region'].value_counts().nlargest(10).plot.bar(figsize=(20, 8), title='Regions with most loss')

#所有负收益产品带来的损失
print('总损失',  loss['Benefit per order'].sum())

#负收益，可能是欺诈交易，来自于哪种支付方式
data[data['Order Status']=='SUSPECTED_FRAUD']['Type'].value_counts() #发现只有TRANSFER

#显示不同地区的欺诈交易情况
high_fraud=data[data['Order Status']=='SUSPECTED_FRAUD']
high_fraud['Order Region'].value_counts().plot.bar(figsize=(20, 8))
plt.title('Regions with Highest Fraud')
plt.ylabel('Fraud Number')
plt.show()

#Western Europe地区欺诈交易最多，将一个bar分成2个部分(堆积条形图)
high_fraud_total=data[data['Order Status']=='SUSPECTED_FRAUD']
high_fraud_we=data[(data['Order Status']=='SUSPECTED_FRAUD']) & (data['Order Status']=='Western Europe')]
#找出风险最高的10个Category
fraud1=high_fraud_total['Category Name'].value_counts().nlargest(10).plot.bar(figsize=(20, 8), title='Fraud Category', color='orange')
fraud2=high_fraud_we['Category Name'].value_counts().nlargest(10).plot.bar(figsize=(20, 8), title='Fraud Category in Western Europe', color='green')
plt.title('Top10 Categories with highest fraud')
plt.xlabel('Products')
plt.show()

#筛选出Top10 Customer(风险)
cus=data[data['Order Status']=='SUSPECTED_FRAUD']
cus['Customer Full Name'].value_counts().nlargest(10).plot.bar(figsize=(20, 8), title='Top10 Highest Fraud Customers') #发现最高Customer Name是MarySmith

#找到MarySmith的交易金额和疑似欺诈的交易金额
print(data[data['Customer Full Name']=='MarySmith']['Sales'].sum())
print(data[(data['Customer Full Name']=='MarySmith') & (data['Order Status']=='SUSPECTED_FRAUD')]['Sales'].sum())

import pickle
with open('data.pkl', 'wb') as file: #将数据保存为pickle，读取比csv快
    pickle.dump(data, file)

import pickle
with open('data.pkl', 'rb') as file: #读取pickle
    train_data=pickle.load(file)

import numpy as np
train_data['fraud']=np.where(train_data['Order Status']=='SUSPECTED_FRAUD', 1, 0)
train_data['fraud'].sum() #总计有多少笔欺诈行为
train_data['late_delivery']=np.where(train_data['Delivery Status']=='Late delivery', 1, 0)
train_data.info()
categorical_cols=train_data.select_dtypes(include='object').columns
for column in train_data.columns: #看哪些字段只有1个取值或所有取值都不同
    if (len(train_data[column].value_counts())<2) or (len(train_data[column].value_counts())=len(train_data)):
        print(column)
train_data.drop(['Customer Email', 'Customer Password', 'Product Description', 'Product Status', 'Product Image', 'Customer Lname', 'Customer Fname'], axis=1, inplace=True)
train_data.drop(['Order Customer Id', 'Order Item Cardprod Id', 'Order Item Id', 'Sales per customer', 'Order Item Total', 'Order Profit Per Order', 'Product Card Id', 'Product Category Id', 'Product Price'], axis=1, inplace=True) #去掉相关性热力图中相关性高的字段
train_data.drop(['Order Status', 'Delivery Status', 'Late_delivery_risk'], axis=1, inplace=True) #去掉会引起标签泄露的字段(预测时把标签放进了特征会导致预测全满分)

#显示所有列
pd.set_option('display.max_columns', None)

train_data.drop(['Order Zipcode', 'shipping data (DateOrders)', 'Latitude', 'Longitude', 'Customer Street', 'order date (DateOrders)', 'order_month_year'], axis=1, inplace=True) #去掉空值过多的字段、时间字段如果不diff也可以去掉、经纬度如果觉得没啥用也可以考虑去掉，训练集测试集分布不一致的字段可以考虑去掉
train_data.info()
train_data=train_data.dropna(subset=['Customer Full Name']) #去掉该字段中的为空的行

#对类别变量LabelEncoder
categorical_cols=train_data.select_dtypes(include='object').columns
from sklearn.preprocessing import LabelEncoder #LabelEncoder不允许有空值
le=LabelEncoder()
for cat in categorical_cols:
    train_data[cat]=le.fit_transform(train_data[cat])
train_data[categorical_cols]

#对数值类型变量
numerical_columns=train_data.columns.tolist()
for x in categorical_cols.tolist():
    numerical_columns.remove(x)

x_fraud=train_data.loc[:, train_data.columns != 'fraud']
y_fraud=train_data['fraud']
x_late=train_data.loc[:, train_data.columns != 'late_delivery']
y_late=train_data['late_delivery']

from sklearn.model_selection import train_test_split
#数据集切分
x_fraud_train, x_fraud_test, y_fraud_train, y_fraud_test=train_test_split(x_fraud, y_fraud, test_size=0.2)
x_late_train, x_late_test, y_late_train, y_late_test=train_test_split(x_late, y_late, test_size=0.2)

from sklearn.preprocessing import StandardScaler
#数据规范化(分类函数如果用到了关于距离的定义则需要规范化，决策树不需要规范化(根据信息增益ID3，信息增益率C4.5和基尼系数而不是按距离分类))
sc=StandardScaler()
x_fraud_train=sc.fit_transform(x_fraud_train)
x_fraud_test=sc.transform(x_fraud_test)
x_late_train=sc.fit_transform(x_late_train)
x_late_test=sc.transform(x_late_test)

accuracy_list={}
recall_list={}
auc_list={}
f1_list={}
from sklearn.metrics import accuracy_score, recall_score, roc_auc_score, confusion_matrix, f1_score
def model_stats(model, x_train, x_test, y_train, y_test, name='Fraud'):
    model=model.fit(x_train, y_train)
    y_pred=model.predict(x_test)
    accuracy=accuracy_score(y_pred, y_test)
    recall=recall_score(y_pred, y_test)
    auc=roc_auc_score(y_pred, y_test)
    f1=f1_score(y_pred, y_test)

    accuracy_list[name, model]=accuracy
    recall_list[name, model]=recall
    auc_list[name, model]=auc
    f1_list[name, model]=f1

    confusion=confusion_matrix(y_pred, y_test)
    print('Model Used: ', model)
    print('{} Accuracy: {}%'.format(name, accuracy*100))
    print('{} Recall: {}%'.format(name, recall*100))
    print('{} AUC: {}%'.format(name, auc*100))
    print('{} F1 Score: {}%'.format(name, f1*100))
    print('{} Confusion Matrix: \n{}'.format(name, confusion))
    return accuracy, recall, f1

from sklearn.linear_model import LogisticRegression
#逻辑回归模型
model_fraud=LogisticRegression()
model_late=LogisticRegression()
model_stats(model_fraud, x_fraud_train, x_fraud_test, y_fraud_train, y_fraud_test, 'Fraud')
model_stats(model_late, x_late_train, x_late_test, y_late_train, y_late_test, 'Late Delivery')

from sklearn.naive_bayes import GaussianNB, BernoulliNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier, DecisionTreeRegressor
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor, GradientBoostingRegressor
from sklearn import svm
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
import xgboost as xgb
#高斯朴素贝叶斯
model_fraud=GaussianNB()
model_late=GaussianNB()
#贝努利朴素贝叶斯
model_fraud=BernoulliNB()
model_late=BernoulliNB()
#SVM
model_fraud=svm.LinearSVC()
model_late=svm.LinearSVC()
#决策树模型和feature importance可视化
model_fraud=DecisionTreeClassifier()
model_late=DecisionTreeClassifier()
import warnings
warnings.filterwarnings('ignore')
model_fraud.feature_importances_ #根据决策树模型，得到feature importance
important_col=model_fraud.feature_importances_.argsort() #得到feature importance的对应列号
print(important_col)
feat_importance=pd.DataFrame({'features': x_fraud.columns[important_col], 'importance': model_fraud.feature_importances_[important_col]})
import seaborn as sns
import matplotlib.pyplot as plt
feat_importance.sort_values(by='importance', ascending=False)
sns.catplot(x='features', y='importance', data=feat_importance, kind='bar', height=5, aspect=2)
plt.xticks(rotation=90)
#KNN模型
model_fraud=KNeighborsClassifier(n_neighbors=1)
model_late=KNeighborsClassifier()
#LDA模型
model_fraud=LinearDiscriminantAnalysis()
model_late=LinearDiscriminantAnalysis()
#随机森林模型
model_fraud=RandomForestClassifier()
model_late=RandomForestClassifier()
#XGBoost分类
model_fraud=xgb.XGBClassifier()
model_late=xgb.XGBClassifier()

#模型评估
model_stats(model_fraud, x_fraud_train, x_fraud_test, y_fraud_train, y_fraud_test, 'Fraud')
model_stats(model_late, x_late_train, x_late_test, y_late_train, y_late_test, 'Late Delivery')

#神经网络进行分类
import tensorflow.keras as keras
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense

keras.layers.BatchNormalization()
classifier=Sequential()
#第1层隐藏层
classifier.add(Dense(1024, activation='relu', kernel_initializer='random_normal', input_dim=x_fraud_train.shape[1]))
#第2层隐藏层
classifier.add(Dense(512, activation='relu', kernel_initializer='random_normal'))
#第3层隐藏层
classifier.add(Dense(256, activation='relu', kernel_initializer='random_normal'))
#第4层隐藏层
classifier.add(Dense(128, activation='relu', kernel_initializer='random_normal'))
#第5层隐藏层
classifier.add(Dense(64, activation='relu', kernel_initializer='random_normal'))
#第6层隐藏层
classifier.add(Dense(32, activation='relu', kernel_initializer='random_normal'))
#第7层隐藏层
classifier.add(Dense(16, activation='relu', kernel_initializer='random_normal'))
#第8层隐藏层
classifier.add(Dense(8, activation='relu', kernel_initializer='random_normal'))
#第9层隐藏层
classifier.add(Dense(4, activation='relu', kernel_initializer='random_normal'))
#第10层隐藏层
classifier.add(Dense(2, activation='relu', kernel_initializer='random_normal'))
#输出层
classifier.add(Dense(1, activation='sigmoid', kernel_initializer='random_normal'))
#定义优化器、损失函数
classifier.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
#训练
classifier.fit(x_fraud_train, y_fraud_train, batch_size=512, epochs=10)
#评估
train_evaluate=classifier.evaluate(x_fraud_train, y_fraud_train)
test_evaluate=classifier.evaluate(x_fraud_test, y_fraud_test)
print('训练集准确率：', train_evaluate)
print('测试集准确率：', test_evaluate)

#模型融合
from sklearn.ensemble import VotingClassifier
from sklearn.model_selection import train_test_split, cross_val_score
#硬投票
ensemble_model=VotingClassifier(estimators=[('LR', model_fraud_lr), ('DT', model_fraud_dt), ('SVC', model_fraud_svc)], voting='hard')
for model, label in zip([model_fraud_lr, model_fraud_dt, model_fraud_svc, ensemble_model], ['LR', 'DT', 'SVC', 'Voting']):
    #scores=cross_val_score(model, x_fraud, y_fraud, cv=5, scoring='accuracy')
    #scores=cross_val_score(model, x_fraud, y_fraud, cv=5, scoring='auc_roc')
    scores=cross_val_score(model, x_fraud, y_fraud, cv=5, scoring='f1')
    print('F1: {:0.2f} (+/- {:0.2f}) [{}]'.format(scores.mean(), scores.std(), label))

#对于销售额进行预测，即Sales字段
#对于订货数量进行预测，即Order Item Quantity
x_sales=train_data.loc[:, train_data.columns !='Sales']
y_sales=train_data['Sales']

x_quantity=train_data.loc[:, train_data.columns !='Order Item Quantity']
y_quantity=train_data['Order Item Quantity']

#数据集切分
x_sales_train, x_sales_test, y_sales_train, y_sales_test=train_test_split(x_sales, y_sales, test_size=0.2)
x_quantity_train, x_quantity_test, y_quantity_train, y_quantity_test=train_test_split(x_quantity, y_quantity, test_size=0.2)

from sklearn.metrics import mean_absolute_error, mean_squared_error
#回归模型，采用MSE, MAE, RMSE指标
def regression_model_stats(model, x_train, x_test, y_train, y_test, model_name='Sales'):
    model=model.fit(x_train, y_train)
    y_pred=model.predict(x_test)
    print('Model Used:', model)
    mae=mean_absolute_error(y_test, y_pred)
    mse=mean_squared_error(y_test, y_pred)
    print('{} MAE {}'.format(model_name, mae))
    print('{} MSE {}'.format(model_name, mse))
    return mae, mse

from sklearn.linear_model import LinearRegression, Lasso, Ridge
import lightgbm as lgb
#Linear Regression
model_sales=LinearRegression()
model_quantitiy=LinearRegression()
#Lasso Regression
model_sales=Lasso()
model_quantitiy=Lasso()
#Ridge Regression
model_sales=Ridge()
model_quantitiy=Ridge()
#Decision Tree Regressor
model_sales=DecisionTreeRegressor()
model_quantitiy=DecisionTreeRegressor()
#XGBoost回归
model_sales=xgb.XGBRegressor()
model_quantitiy=xgb.XGBRegressor()
#LightGBM回归
model_sales=lgb.LGBMRegressor()
model_quantitiy=lgb.LGBMRegressor()
#随机森林
model_sales=RandomForestRegressor()
model_quantitiy=RandomForestRegressor()

#模型评估
regression_model_stats(model_sales, x_sales_train, x_sales_test, y_sales_train, y_sales_test, model_name='Sales')
regression_model_stats(model_quantitiy_sales, x_quantitiy_train, x_quantitiy_test, y_quantitiy_train, y_quantitiy_test, model_name='Quantity')